In [0]:
from pyspark.sql.functions import *

silver_df = spark.table("config_catalog.silver.customers")

gold_df = silver_df.selectExpr(
    "uuid() as customer_sk",
    "customer_id",
    "customer_name",
    "city",
    "last_updated",
    "current_timestamp() as valid_from",
    "CAST(NULL AS TIMESTAMP) as valid_to",
    "true as is_current"
)

gold_df.write.format("delta").mode("append").saveAsTable("config_catalog.gold.dim_customers")

In [0]:
silver_df = spark.table("config_catalog.silver.customers")

In [0]:
from delta.tables import DeltaTable

gold_delta = DeltaTable.forName(spark, "config_catalog.gold.dim_customers")
gold_df = gold_delta.toDF()

gold_current = gold_df.filter("is_current = true")

In [0]:
joined_df = silver_df.alias("src").join(
    gold_current.alias("tgt"),
    "customer_id",
    "left"
)

changed_df = joined_df.filter("""
    tgt.customer_id IS NOT NULL AND (
        src.customer_name <> tgt.customer_name OR
        src.city <> tgt.city OR
        src.last_updated <> tgt.last_updated
    )
""")

In [0]:
changed_df = joined_df.filter(
    (col("tgt.customer_id").isNotNull()) &
    (
        (col("src.customer_name") != col("tgt.customer_name")) |
        (col("src.city") != col("tgt.city"))
    )
)

In [0]:
from pyspark.sql.window import Window
from pyspark.sql.functions import row_number, col

window_spec = Window.partitionBy("customer_id").orderBy(col("src.last_updated").desc())

dedup_df = changed_df.withColumn(
    "rn",
    row_number().over(window_spec)
).filter(col("rn") == 1).drop("rn")

In [0]:
gold_delta.alias("tgt").merge(
    dedup_df.alias("src"),   # ✅ USE dedup_df
    "tgt.customer_id = src.customer_id AND tgt.is_current = true"
).whenMatchedUpdate(set={
    "valid_to": current_timestamp(),
    "is_current": lit(False)
}).execute()

In [0]:
from pyspark.sql.functions import col

clean_dedup_df = dedup_df.select(
    col("src.customer_id").alias("customer_id"),
    col("src.customer_name").alias("customer_name"),
    col("src.city").alias("city"),
    col("src.last_updated").alias("last_updated")
)

In [0]:
insert_df = clean_dedup_df.selectExpr(
    "uuid() as customer_sk",
    "customer_id",
    "customer_name",
    "city",
    "last_updated",
    "current_timestamp() as valid_from",
    "CAST(NULL AS TIMESTAMP) as valid_to",
    "true as is_current"
)

insert_df.write.format("delta").mode("append").saveAsTable("config_catalog.gold.dim_customers")

In [0]:
dbutils.widgets.text("run_id", "")
run_id = dbutils.widgets.get("run_id")

dbutils.widgets.text("pipeline_name", "")
pipeline_name = dbutils.widgets.get("pipeline_name")



jdbc_url = "jdbc:sqlserver://configserver18.database.windows.net:1433;database=config_db"

connection_props = {
    "user": 'Mahi_123',
    "password":'Maram_098',
    "driver": "com.microsoft.sqlserver.jdbc.SQLServerDriver"
}
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, TimestampType

from pyspark.sql.types import *
from pyspark.sql import Row
from datetime import datetime
from pyspark.sql.functions import col

schema = StructType([
    StructField("pipeline_name", StringType(), True),
    StructField("run_id", StringType(), True),
    StructField("dataset_name", StringType(), True),
    StructField("status", StringType(), True),
    StructField("rows_copied", IntegerType(), True),
    StructField("error_message", StringType(), True),
    StructField("end_time", TimestampType(), True)
])



success_row = [Row(
        pipeline_name=pipeline_name,
        run_id=run_id,
        dataset_name="DIM_CUSTOMERS",
        status="SUCCESS",
        rows_copied=int(rows_copied),
        error_message=None,
        end_time=datetime.now()
    )]

    success_df = spark.createDataFrame(success_row, schema)
    success_df.write.jdbc(url=jdbc_url, table="pipeline_log", mode="append", properties=connection_props)